In [15]:
import os
import glob
import pandas as pd
import pymongo
import json
import numpy as np

## Extract Stocks

In [2]:
#Get list of stock names
os.listdir("../etl-project/RawStock")
stock_filename = os.listdir("../etl-project/RawStock")


In [3]:
stock_data = [pd.read_csv(f'../etl-project/RawStock/{file}') for file in stock_filename]

In [4]:
print(stock_filename)

['ibm.us.txt', 'ko.us.txt', 'intc.us.txt', 'wmt.us.txt', 'mcd.us.txt', 'nke.us.txt', 'msft.us.txt', 'xom.us.txt', 'ba.us.txt', 'jpm.us.txt', 'wba.us.txt', 'v.us.txt', 'utx.us.txt', 'dis.us.txt', 'aapl.us.txt', 'mmm.us.txt', 'mrk.us.txt', 'gs.us.txt', 'unh.us.txt', 'hd.us.txt', 'jnj.us.txt', 'csco.us.txt', 'cvx.us.txt', 'vz.us.txt', 'cat.us.txt', 'trv.us.txt', 'axp.us.txt', 'pg.us.txt', 'pfe.us.txt']


## Transform Stocks 

In [5]:
clean_ticker = [ticker[:-7] for ticker in stock_filename]
clean_ticker.sort()


['aapl',
 'axp',
 'ba',
 'cat',
 'csco',
 'cvx',
 'dis',
 'gs',
 'hd',
 'ibm',
 'intc',
 'jnj',
 'jpm',
 'ko',
 'mcd',
 'mmm',
 'mrk',
 'msft',
 'nke',
 'pfe',
 'pg',
 'trv',
 'unh',
 'utx',
 'v',
 'vz',
 'wba',
 'wmt',
 'xom']

In [6]:
stock_company = {
    "Ticker":clean_ticker,
    "Company Name": ["Apple","American Express","Boeing","Caterpillar","Cisco","Chevron","Disney","Goldman Sachs","Home Depot","IBM","Intel","Johnson & Johnson","JP Morgan Chase","Coca-Cola","McDonalds","3M","Merck","Microsoft","Nike","Pfizer","Proctor & Gamble","Travelers Companies","United Health","United Technologies","Visa","Verizon","Walgreens","Walmart","Exxon Mobil"]
}

stock_company = pd.DataFrame(stock_company)

In [7]:
for x in np.arange(len(stock_data)):
    stock_data[x]["Ticker"] = clean_ticker[x]

In [8]:
stocks = pd.concat(stock_data, axis=0)

In [9]:
stocks = stocks[["Ticker","Date","Open","Close"]]

In [10]:
stocks["Daily Percent"] = ((stocks["Close"]-stocks["Open"])/stocks["Open"])*100

In [11]:
stocks = pd.merge(stocks,stock_company, on="Ticker")

In [12]:
stocks.sample(20)

,Ticker,Date,Open,Close,Daily Percent,Company Name
19593,axp,1991-11-25,7.01110,6.98260,-0.406498,American Express
7605,aapl,1992-03-31,17.34400,17.36900,0.144142,Apple
180609,msft,2000-08-22,105.85000,107.57000,1.624941,Microsoft
205782,pg,1988-11-08,3.51590,3.52410,0.233226,Proctor & Gamble
193029,pfe,1986-03-04,0.27487,0.27487,0.000000,Pfizer
123604,jnj,2017-10-18,107.62000,107.80000,0.167255,Johnson & Johnson
32268,ba,1996-04-30,6.77640,6.76010,-0.240541,Boeing
263294,wba,2001-01-18,40.16800,39.38000,-1.961761,Walgreens
98838,hd,2009-06-12,42.78800,43.36400,1.346172,Home Depot
104261,ibm,1983-01-27,3.23370,3.33380,3.095525,IBM


## Load Stocks

In [13]:
stock_data='stocks.csv'

stocks.to_csv(stock_data)

In [16]:
client = pymongo.MongoClient('localhost', 27017)
mng_db = client['FinanceDB'] 
collection_name = 'Stock_Data' 
db = mng_db[collection_name]

data = pd.read_csv(stock_data)
data_json = json.loads(data.to_json(orient='records'))
db.insert_many(data_json)